<a href="https://colab.research.google.com/github/jeg357/jeg357.github.io/blob/master/AiCan10_29_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%tensorflow_version 2.x
#!pip uninstall -y tensorflow


In [ ]:
!pip install tensorflow-gpu==2.0.0-beta0
!pip install tensorflow_hub


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
pd.set_option("display.precision", 8)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_root='/content/drive/My Drive/Images/Classified_Images'

In [ ]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
print(TRAINING_DATA_DIR);
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset="validation",
shuffle=True,
target_size=IMAGE_SHAPE
)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset="training",
shuffle=True,
target_size=IMAGE_SHAPE)

/content/drive/My Drive/Images/Classified_Images
Found 503 images belonging to 6 classes.
Found 2023 images belonging to 6 classes.


In [ ]:
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 6)
['Cardboard' 'Glass' 'Metal' 'Paper' 'Plastic' 'Trash']


In [ ]:
model = tf.keras.Sequential([
hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
output_shape=[1280],
trainable=False),
tf.keras.layers.Dropout(0.4),
tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])
model.summary()
model.compile(
optimizer=tf.keras.optimizers.Adam(),
loss='categorical_crossentropy',
metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  7686      
Total params: 2,265,670
Trainable params: 7,686
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
hist = model.fit(
train_generator,
epochs=10,
verbose=1,
steps_per_epoch=steps_per_epoch,
validation_data=valid_generator,
validation_steps=val_steps_per_epoch).history

Epoch 1/10


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


64/64 [==============================] - 1160s 18s/step - loss: 1.3214 - acc: 0.3857 - val_loss: 0.8546 - val_acc: 0.6720
Epoch 2/10
64/64 [==============================] - 373s 6s/step - loss: 0.7425 - acc: 0.7196 - val_loss: 0.7581 - val_acc: 0.7157
Epoch 3/10
64/64 [==============================] - 366s 6s/step - loss: 0.5806 - acc: 0.7724 - val_loss: 0.7018 - val_acc: 0.7336
Epoch 4/10
64/64 [==============================] - 354s 6s/step - loss: 0.5211 - acc: 0.8314 - val_loss: 0.7059 - val_acc: 0.7256
Epoch 5/10
64/64 [==============================] - 355s 6s/step - loss: 0.4713 - acc: 0.8299 - val_loss: 0.6829 - val_acc: 0.7396
Epoch 6/10
64/64 [==============================] - 358s 6s/step - loss: 0.4138 - acc: 0.8424 - val_loss: 0.7046 - val_acc: 0.7157
Epoch 7/10
64/64 [==============================] - 392s 6s/step - loss: 0.3876 - acc: 0.8670 - val_loss: 0.6571 - val_acc: 0.7535
Epoch 8/10
64/64 [==============================] - 381s 6s/step - loss: 0.3544 - acc: 0.870

In [ ]:
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

In [ ]:
RECYCLE_SAVED_MODEL = "saved_models/Recycle"
tf.keras.experimental.export_saved_model(model, RECYCLE_SAVED_MODEL)

In [ ]:
recycle_model = tf.keras.experimental.load_from_saved_model(RECYCLE_SAVED_MODEL,
                                                              custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
val_image_batch, val_label_batch = next(iter(valid_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)
print("Validation batch shape:", val_image_batch.shape)